In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.joinpath('src').resolve()))
from utils import *
import copy
import math
from public_tests import *

%matplotlib inline

In [2]:
X_train, y_train = load_data("../data/samples/higgs_train.csv")

In [3]:
print("First five elements in X_train are:\n", X_train[:5])
print("Type of X_train:",type(X_train))

First five elements in X_train are:
          f1        f2        f3        f4        f5        f6        f7  \
0  0.719958  0.785939  0.981966  0.553060  1.383569  0.925050  1.853685   
1  0.673473  0.110004 -1.289741  1.493732  1.224188  1.020413  0.265356   
2  1.228540 -0.038039 -0.914100  0.579205  0.240356  0.813564  0.720862   
3  2.154200  0.142145 -1.209841  1.915257 -1.589158  2.047788 -1.414074   
4  1.502871 -0.042909 -0.124530  2.707755 -0.449995  0.880712 -0.276300   

         f8        f9       f10  ...       f19       f20       f21       f22  \
0 -1.074324  0.000000  0.905498  ...  0.892326  0.797247  3.101961  1.014163   
1 -0.701229  0.000000  0.920984  ... -0.327760 -1.343154  3.101961  1.564466   
2  1.177100  1.086538  1.150884  ...  1.026410  0.569178  1.550981  1.401413   
3  0.177561  0.000000  0.784379  ... -0.152867  1.314980  0.000000  0.916103   
4  1.111683  0.000000  0.771159  ... -0.712525  1.622956  0.000000  0.951986   

        f23       f24       f25

In [4]:
print("First five elements in y_train are:\n", y_train[:5])
print("Type of y_train:",type(y_train))

First five elements in y_train are:
 [1 0 1 1 0]
Type of y_train: <class 'numpy.ndarray'>


In [5]:
print ('The shape of X_train is: ' + str(X_train.shape))
print ('The shape of y_train is: ' + str(y_train.shape))
print ('We have m = %d training examples' % (len(y_train)))

The shape of X_train is: (400000, 28)
The shape of y_train is: (400000,)
We have m = 400000 training examples


In [6]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [7]:
def compute_cost(X, y, w, b, lambda_=0.0):
    m = X.shape[0]
    z = X.dot(w) + b           # shape (m,)
    p = sigmoid(z)             # shape (m,)
    # clip to avoid log(0)
    eps = 1e-12
    cost = - (y * np.log(p + eps) + (1 - y) * np.log(1 - p + eps)).mean()
    # add regularization if desired:
    if lambda_:
        cost += (lambda_ / (2*m)) * np.sum(w**2)
    return cost

In [8]:
m, n = X_train.shape

# Compute and display cost with w initialized to zeroes
initial_w = np.zeros(n)
initial_b = 0.
cost = compute_cost(X_train, y_train, initial_w, initial_b)
print('Cost at initial w (zeros): {:.3f}'.format(cost))

Cost at initial w (zeros): 0.693


In [9]:
def compute_gradient(X, y, w, b, lambda_=0.0):
    m = X.shape[0]
    z = X.dot(w) + b
    p = sigmoid(z)
    error = p - y               # shape (m,)
    dj_dw = (X.T.dot(error) / m) + (lambda_ / m) * w
    dj_db = error.mean()
    return dj_db, dj_dw

In [10]:
# Compute and display gradient with w initialized to zeroes
initial_w = np.zeros(n)
initial_b = 0.

dj_db, dj_dw = compute_gradient(X_train, y_train, initial_w, initial_b)
print(f'dj_db at initial w (zeros):{dj_db}' )
print(f'dj_dw at initial w (zeros):{dj_dw.tolist()}' )

dj_db at initial w (zeros):-0.02879
dj_dw at initial w (zeros):[-0.014671717771850527, 0.00042985919293016196, -0.001118724101784046, 0.00202225888157438, -0.000175184210143032, -0.042020431100856515, -0.0003710050517620857, 0.0005153622108769695, -0.023820989152491093, -0.0345640292387642, -0.0007117815831749067, 0.001275515634009098, -0.0032863165107369415, -0.03209177182596177, -0.001303456493557278, -0.00040129368487921963, -0.014346503648757935, -0.03738590491063893, -0.00042448005668884435, 0.0006411599186694729, -0.037737298844009635, -0.03427435940969736, -0.0342536618277058, -0.03123258922241628, -0.02289227080712095, 0.011407724410700612, -0.017615057053864003, -0.008397993431910873]


In [11]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_): 
    """
    Performs batch gradient descent to learn theta. Updates theta by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X :    (array_like Shape (m, n)
      y :    (array_like Shape (m,))
      w_in : (array_like Shape (n,))  Initial values of parameters of the model
      b_in : (scalar)                 Initial value of parameter of the model
      cost_function:                  function to compute cost
      alpha : (float)                 Learning rate
      num_iters : (int)               number of iterations to run gradient descent
      lambda_ (scalar, float)         regularization constant
      
    Returns:
      w : (array_like Shape (n,)) Updated values of parameters of the model after
          running gradient descent
      b : (scalar)                Updated value of parameter of the model after
          running gradient descent
    """
    
    # number of training examples
    m = len(X)
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w_history = []
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(X, y, w_in, b_in, lambda_)   

        # Update Parameters using w, b, alpha and gradient
        w_in = w_in - alpha * dj_dw               
        b_in = b_in - alpha * dj_db              
       
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            cost =  cost_function(X, y, w_in, b_in, lambda_)
            J_history.append(cost)

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
        
    return w_in, b_in, J_history, w_history #return w and J,w history for graphing

In [15]:
np.random.seed(1)
initial_w = np.random.randn(n) * 0.01  # shape (n,) small random values
initial_b = 0.0

# Some gradient descent settings
iterations = 10000
alpha = 0.005

# Run gradient descent (batch)
w, b, J_history, _ = gradient_descent(X_train, y_train, initial_w, initial_b,
                                           compute_cost, compute_gradient, alpha, iterations, 0)

print('w shape:', w.shape)
print('b:', b)
print('Last cost:', J_history[-1] if len(J_history) > 0 else None)

Iteration    0: Cost     0.69   
Iteration 1000: Cost     0.68   
Iteration 1000: Cost     0.68   
Iteration 2000: Cost     0.67   
Iteration 2000: Cost     0.67   
Iteration 3000: Cost     0.67   
Iteration 3000: Cost     0.67   
Iteration 4000: Cost     0.66   
Iteration 4000: Cost     0.66   
Iteration 5000: Cost     0.66   
Iteration 5000: Cost     0.66   
Iteration 6000: Cost     0.66   
Iteration 6000: Cost     0.66   
Iteration 7000: Cost     0.66   
Iteration 7000: Cost     0.66   
Iteration 8000: Cost     0.65   
Iteration 8000: Cost     0.65   
Iteration 9000: Cost     0.65   
Iteration 9000: Cost     0.65   
Iteration 9999: Cost     0.65   
w shape: (28,)
b: 0.2215982435920402
Last cost: 0.6514710420448836
Iteration 9999: Cost     0.65   
w shape: (28,)
b: 0.2215982435920402
Last cost: 0.6514710420448836


In [17]:
# Diagnostics: check J_history, gradient norms, feature scales and label balance
try:
    print('Length of J_history:', len(J_history))
    print('First 10 costs:', J_history[:10])
except NameError:
    print('J_history not found in this scope. Run the gradient-descent cell first.')

# Current parameters (if available) and gradient norms
try:
    dj_db_curr, dj_dw_curr = compute_gradient(X_train, y_train, w, b)
    print('Current dj_db (scalar):', float(dj_db_curr))
    print('Norm of dj_dw (L2):', float(np.linalg.norm(dj_dw_curr)))
except Exception as e:
    print('Could not compute current gradient:', repr(e))

# Feature scale summary (top 10 largest stds)
try:
    feature_means = X_train.mean(axis=0)
    feature_stds = X_train.std(axis=0)
    top_idx = np.argsort(feature_stds)[-10:][::-1]
    print('Top 10 features by std (index, std):')
    for i in top_idx:
        print(int(i), float(feature_stds[i]))
    print('Max absolute value in X_train:', float(np.abs(X_train).max()))
except Exception as e:
    print('Could not compute feature stats:', repr(e))

# Label distribution
try:
    vals, counts = np.unique(y_train, return_counts=True)
    print('Label distribution (val:count, fraction):')
    for v,c in zip(vals, counts):
        print(int(v), c, c/len(y_train))
except Exception as e:
    print('Could not compute label distribution:', repr(e))

# Quick experiment: standardize features on a subset and re-run gradient descent to compare convergence speed
try:
    from sklearn.preprocessing import StandardScaler
    subset = 50000 if len(y_train) > 50000 else len(y_train)
    print(f'Using subset size={subset} for scaled experiment')
    X_sub = X_train[:subset].astype(float)
    y_sub = y_train[:subset]
    scaler = StandardScaler()
    X_sub_s = scaler.fit_transform(X_sub)
    # initialize small weights
    w0 = np.zeros(n)
    b0 = 0.0
    alpha2 = 0.05
    iters2 = 2000
    print(f'Running gradient descent on scaled subset: alpha={alpha2}, iters={iters2}')
    w2, b2, J2, _ = gradient_descent(X_sub_s, y_sub, w0, b0, compute_cost, compute_gradient, alpha2, iters2, 0)
    print('Scaled subset initial cost:', J2[0], 'final cost:', J2[-1], 'reduction:', J2[0]-J2[-1])
except Exception as e:
    print('Scaled experiment skipped or failed (sklearn may be missing):', repr(e))

Length of J_history: 10000
First 10 costs: [np.float64(0.693932819724718), np.float64(0.6938378641335086), np.float64(0.6937463780701792), np.float64(0.6936582030857349), np.float64(0.69357318800832), np.float64(0.6934911886112227), np.float64(0.6934120672957609), np.float64(0.6933356927884076), np.float64(0.6932619398515436), np.float64(0.6931906890072443)]
Current dj_db (scalar): -0.002110130677671557
Norm of dj_dw (L2): 0.013215621673833323
Top 10 features by std (index, std):
20 1.3997997954168464
16 1.1940273109730548
12 1.049895860655212
8 1.0275673890706865
1 1.0091470710892596
14 1.0088198543150915
6 1.0086747337804196
18 1.007825876768817
10 1.00773446777511
2 1.0068005887708906
Could not compute feature stats: TypeError("cannot convert the series to <class 'float'>")
Label distribution (val:count, fraction):
0 188484 0.47121
1 211516 0.52879
Using subset size=50000 for scaled experiment
Running gradient descent on scaled subset: alpha=0.05, iters=2000
Iteration    0: Cost    

/var/folders/f_/ps1v2btn63d27h0glvzpqshm0000gn/T/ipykernel_66737/3165227829.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(int(i), float(feature_stds[i]))


Iteration  200: Cost     0.65   
Iteration  400: Cost     0.65   
Iteration  400: Cost     0.65   
Iteration  600: Cost     0.64   
Iteration  600: Cost     0.64   
Iteration  800: Cost     0.64   
Iteration  800: Cost     0.64   
Iteration 1000: Cost     0.64   
Iteration 1000: Cost     0.64   
Iteration 1200: Cost     0.64   
Iteration 1200: Cost     0.64   
Iteration 1400: Cost     0.64   
Iteration 1400: Cost     0.64   
Iteration 1600: Cost     0.64   
Iteration 1600: Cost     0.64   
Iteration 1800: Cost     0.64   
Iteration 1800: Cost     0.64   
Iteration 1999: Cost     0.64   
Scaled subset initial cost: 0.6922998307981906 final cost: 0.6398136646451976 reduction: 0.05248616615299295
Iteration 1999: Cost     0.64   
Scaled subset initial cost: 0.6922998307981906 final cost: 0.6398136646451976 reduction: 0.05248616615299295


In [18]:
def predict(X, w, b): 
    """
    Predict whether the label is 0 or 1 using learned logistic
    regression parameters w
    
    Args:
    X : (ndarray Shape (m, n))
    w : (array_like Shape (n,))      Parameters of the model
    b : (scalar, float)              Parameter of the model

    Returns:
    p: (ndarray (m,1))
        The predictions for X using a threshold at 0.5
    """
    # number of training examples
    m, n = X.shape   
    p = np.zeros(m)
   
    ### BEGIN CODE HERE
    fw = sigmoid(np.dot(X, w) + b)
    p = (fw >= 0.5).astype(int)
    
    
    ### END CODE
    return p

In [19]:
X_test, y_test = load_data("../data/samples/higgs_test.csv")

In [21]:
p = predict(X_test, w,b)
print('Train Accuracy: %f'%(np.mean(p == y_test) * 100))

Train Accuracy: 61.958000


In [ ]:
# Evaluation: compute common metrics and run a small sklearn baseline on a subset
print('Preparing evaluation...')
# load validation set if present
try:
    X_val, y_val = load_data('../data/samples/higgs_val.csv')
    print('Loaded validation set:', X_val.shape, y_val.shape)
except Exception as e:
    X_val, y_val = None, None
    print('Validation set not available or failed to load:', repr(e))

# Utility: safe metrics display
def print_metrics(name, y_true, y_pred, y_prob=None):
    try:
        from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
    except Exception as e:
        print('sklearn.metrics unavailable:', repr(e))
        print(name, 'accuracy (fallback):', float((y_true==y_pred).mean()))
        return
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    print(f'-- {name} --')
    print(f'Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}')
    try:
        if y_prob is not None:
            auc = roc_auc_score(y_true, y_prob)
            print(f'ROC AUC: {auc:.4f}')
    except Exception as e:
        print('ROC AUC failed:', repr(e))
    print('Confusion matrix:')
    print(confusion_matrix(y_true, y_pred))

# Predictions and metrics for train/val/test (use predict/probabilities)
try:
    # Ensure w,b exist
    _ = w
    _ = b
    p_train = predict(X_train, w, b)
    probs_train = sigmoid(X_train.dot(w) + b)
    print_metrics('Train', y_train, p_train, probs_train)
except Exception as e:
    print('Train evaluation failed:', repr(e))

try:
    X_test, y_test = load_data('../data/samples/higgs_test.csv')
    p_test = predict(X_test, w, b)
    probs_test = sigmoid(X_test.dot(w) + b)
    print_metrics('Test', y_test, p_test, probs_test)
except Exception as e:
    print('Test evaluation failed:', repr(e))

if X_val is not None and y_val is not None:
    try:
        p_val = predict(X_val, w, b)
        probs_val = sigmoid(X_val.dot(w) + b)
        print_metrics('Validation', y_val, p_val, probs_val)
    except Exception as e:
        print('Validation evaluation failed:', repr(e))

# Baseline: sklearn LogisticRegression on a subset (standardized)
try:
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import StandardScaler
    subset = min(100000, len(y_train))
    print(f'Fitting sklearn LogisticRegression on subset={subset} (this may take a moment)')
    X_sub = X_train[:subset].astype(float)
    y_sub = y_train[:subset]
    scaler = StandardScaler().fit(X_sub)
    X_sub_s = scaler.transform(X_sub)
    clf = LogisticRegression(solver='saga', max_iter=200, n_jobs=-1, C=1.0)
    clf.fit(X_sub_s, y_sub)
    # evaluate on a scaled test subset
    X_test_s = scaler.transform(X_test[:subset].astype(float)) if 'X_test' in globals() else None
    y_test_sub = y_test[:subset] if 'y_test' in globals() else None
    if X_test_s is not None:
        preds = clf.predict(X_test_s)
        probs = clf.predict_proba(X_test_s)[:,1]
        print_metrics('sklearn LogisticRegression (subset test)', y_test_sub, preds, probs)
    else:
        print('sklearn baseline fitted but no test data available to evaluate on subset')
except Exception as e:
    print('sklearn baseline skipped or failed:', repr(e))


Preparing evaluation...
Loaded validation set: (50000, 28) (50000,)
-- Train --
Accuracy: 0.6184, Precision: 0.6192, Recall: 0.7230, F1: 0.6671
ROC AUC: 0.6598
Confusion matrix:
[[ 94430  94054]
 [ 58593 152923]]
-- Train --
Accuracy: 0.6184, Precision: 0.6192, Recall: 0.7230, F1: 0.6671
ROC AUC: 0.6598
Confusion matrix:
[[ 94430  94054]
 [ 58593 152923]]
-- Test --
Accuracy: 0.6196, Precision: 0.6197, Recall: 0.7260, F1: 0.6687
ROC AUC: 0.6606
Confusion matrix:
[[11783 11778]
 [ 7243 19196]]
-- Validation --
Accuracy: 0.6179, Precision: 0.6193, Recall: 0.7202, F1: 0.6659
ROC AUC: 0.6595
Confusion matrix:
[[11852 11708]
 [ 7397 19043]]
Fitting sklearn LogisticRegression on subset=100000 (this may take a moment)
-- Test --
Accuracy: 0.6196, Precision: 0.6197, Recall: 0.7260, F1: 0.6687
ROC AUC: 0.6606
Confusion matrix:
[[11783 11778]
 [ 7243 19196]]
-- Validation --
Accuracy: 0.6179, Precision: 0.6193, Recall: 0.7202, F1: 0.6659
ROC AUC: 0.6595
Confusion matrix:
[[11852 11708]
 [ 7397 1